# Problem: Optimizing waste management

In [1]:
using JuMP
using Gurobi
using CSV
using DataFrames

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-19


### Data importation

In [3]:
centers = CSV.File("HW3 data/centers.csv",header=0) |> Tables.matrix;
stations = CSV.File("HW3 data/stations.csv",header=0) |> Tables.matrix;
landfills = CSV.File("HW3 data/landfills.csv",header=0) |> Tables.matrix;
q = CSV.File("HW3 data/q.csv",header=0) |> Tables.matrix;

centers2 = CSV.File("HW3 data/centers2.csv",header=0) |> Tables.matrix;
stations2 = CSV.File("HW3 data/stations2.csv",header=0) |> Tables.matrix;
landfills2 = CSV.File("HW3 data/landfills2.csv",header=0) |> Tables.matrix;
q2 = CSV.File("HW3 data/q2.csv",header=0) |> Tables.matrix;

centers_all = [centers;centers2];
stations_all = [stations;stations2];
landfills_all = [landfills;landfills2];
q_all = [q;q2];

n1 = size(centers)[1]
s1 = size(stations)[1]
m1 = size(landfills)[1]
n2 = size(centers2)[1]
s2 = size(stations2)[1]
m2 = size(landfills2)[1]
n_all = n1+n2;
s_all = s1+s2;
m_all = m1+m2;

## Part A. Building landfills
### Part a)

In [4]:
# Euclidean distance function
function distance(p1, p2)
    return sqrt((p1[1] - p2[1])^2 + (p1[2] - p2[2])^2)
end

# Distance matrix between centers and landfills
d = [distance(centers[i, :], landfills[j, :]) for i in 1:n1, j in 1:m1]

# Create the optimization model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[j=1:m1], Bin)           # 1 if landfill j is built
@variable(model, y[i=1:n1, j=1:m1] >= 0)   # Amount of waste from center i to landfill j (tons per day)

# Objective: Minimize total transportation cost (distance * waste)
@objective(model, Min, sum(y[i,j] * d[i,j] for i in 1:n1, j in 1:m1))

# Constraints

# All waste from each center must be assigned to landfills
for i in 1:n1
    @constraint(model, sum(y[i,j] for j in 1:m1) == q[i])
end

# Waste can only be sent to built landfills
for i in 1:n1, j in 1:m1
    @constraint(model, y[i,j] <= q[i] * x[j])
end

# Exactly 5 landfills must be built
@constraint(model, sum(x[j] for j in 1:m1) == 5)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-19


x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] = 5

In [5]:
# Solve the model
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 801 rows, 765 columns and 2265 nonzeros
Model fingerprint: 0xdee7c899
Variable types: 750 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+03]
Presolve time: 0.01s
Presolved: 801 rows, 765 columns, 2265 nonzeros
Variable types: 750 continuous, 15 integer (15 binary)
Found heuristic solution: objective 1026891.5990

Root relaxation: objective 8.404875e+05, 132 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    840487.52400 840487.524  0.00%     -    0s

Explored 1 nodes (13

### Part b)

In [6]:
# Output results
println("Objective value (total distance traveled): ", objective_value(model))

# Extract the values of x
x_vals = value.(x)

# Landfills that are built
built_landfills = [j for j in 1:m1 if x_vals[j] > 0.5]  # Using a threshold to account for numerical precision

println("Landfills built at locations: ", built_landfills)

Objective value (total distance traveled): 840487.5240012797
Landfills built at locations: [3, 4, 7, 13, 14]


## Part B. Building landfills and transfer stations
### Part c)

In [7]:
# Distance matrices
d_center_landfill = [distance(centers[i, :], landfills[j, :]) for i in 1:n1, j in 1:m1]
d_center_station = [distance(centers[i, :], stations[k, :]) for i in 1:n1, k in 1:s1]
d_station_landfill = [distance(stations[k, :], landfills[j, :]) for k in 1:s1, j in 1:m1]

# Total waste generated
total_waste = sum(q[i] for i in 1:n1)

# Create the optimization model
model1 = Model(Gurobi.Optimizer)

# Decision variables
@variable(model1, x[j=1:m1], Bin)  # 1 if landfill j is built
@variable(model1, z[k=1:s1], Bin)  # 1 if transfer station k is built
@variable(model1, y[i=1:n1, j=1:m1] >= 0)  # Waste from center i to landfill j
@variable(model1, w[i=1:n1, k=1:s1] >= 0)  # Waste from center i to transfer station k
@variable(model1, t[k=1:s1, j=1:m1] >= 0)  # Waste from transfer station k to landfill j

# Objective: Minimize total cost
@objective(model1, Min,
    # Direct transport from centers to landfills
    sum(y[i,j] * d_center_landfill[i,j] * 1.0 for i in 1:n1, j in 1:m1) +
    # Transport from centers to transfer stations
    sum(w[i,k] * d_center_station[i,k] * 1.0 for i in 1:n1, k in 1:s1) +
    # Transport from transfer stations to landfills
    sum(t[k,j] * d_station_landfill[k,j] * 0.5 for k in 1:s1, j in 1:m1) +
    # Operating cost of transfer stations
    sum(10000 * z[k] for k in 1:s1)
)

# Constraints
for i in 1:n1
    # All waste from center i must go to either a landfill or a station
    @constraint(model1, sum(y[i,j] for j in 1:m1) + sum(w[i,k] for k in 1:s1) == q[i])
end

for k in 1:s1
    # Capacity constraint for transfer stations
    @constraint(model1, sum(w[i,k] for i in 1:n1) <= 2000 * z[k])
end

for k in 1:s1
    # Waste must reach a landfill if it is sent to a transfer station
    @constraint(model1, sum(t[k,j] for j in 1:m1) == sum(w[i,k] for i in 1:n1))
end

for j in 1:m1
    # Landfills can only receive waste if built
    @constraint(model1, sum(y[i,j] for i in 1:n1) + sum(t[k,j] for k in 1:s1) <= total_waste * x[j])
end

# Exactly 5 landfills must be built
@constraint(model1, sum(x[j] for j in 1:m1) <= 5)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-19


x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] ≤ 5

In [8]:
# Solve the model
optimize!(model1)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 166 rows, 4065 columns and 10580 nonzeros
Model fingerprint: 0xa4f50328
Variable types: 4000 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+04]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+03]
Presolve time: 0.01s
Presolved: 166 rows, 4065 columns, 10580 nonzeros
Variable types: 4000 continuous, 65 integer (65 binary)
Found heuristic solution: objective 1091974.8279

Root relaxation: objective 6.681715e+05, 117 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 668171.460    0   14 1091974.83 668171.460  38.8%     -    0s
H    0     0   

### Part d)

In [9]:

# Output results
println("Objective value: ", objective_value(model1))

# Get the indices of the landfills and transfer stations that are built
built_landfills = []
for j in 1:m1
    if value(x[j]) > 0.5
        push!(built_landfills, j)
    end
end
println("Landfills built at indices: ", built_landfills)

built_stations = []
for k in 1:s1
    if value(z[k]) > 0.5
        push!(built_stations, k)
    end
end
println("Transfer stations built at indices: ", built_stations)

Objective value: 808642.7541913455
Landfills built at indices: Any[3, 4, 7, 13, 14]
Transfer stations built at indices: Any[4, 12, 21, 23, 30, 31, 32, 39, 45]


In [10]:
# Extract variable values
y_vals = value.(y)
w_vals = value.(w)
t_vals = value.(t)

# Calculate distances
distance_direct = sum(y_vals[i,j] * d_center_landfill[i,j] for i in 1:n1, j in 1:m1)
distance_to_stations = sum(w_vals[i,k] * d_center_station[i,k] for i in 1:n1, k in 1:s1)
distance_from_stations = sum(t_vals[k,j] * d_station_landfill[k,j] for k in 1:s1, j in 1:m1)

total_distance = distance_direct + distance_to_stations + distance_from_stations

println("Distance traveled directly from centers to landfills (miles-tons): ", distance_direct)
println("Distance traveled from centers to transfer stations (miles-tons): ", distance_to_stations)
println("Distance traveled from transfer stations to landfills (miles-tons): ", distance_from_stations)
println("Total distance traveled by waste (miles-tons): ", total_distance)

Distance traveled directly from centers to landfills (miles-tons): 404537.24764089426
Distance traveled from centers to transfer stations (miles-tons): 156251.50414975767
Distance traveled from transfer stations to landfills (miles-tons): 315708.00480138697
Total distance traveled by waste (miles-tons): 876496.7565920389


In [11]:
# Calculate transportation costs
cost_direct = sum(y_vals[i,j] * d_center_landfill[i,j] * 1.0 for i in 1:n1, j in 1:m1)
cost_to_stations = sum(w_vals[i,k] * d_center_station[i,k] * 1.0 for i in 1:n1, k in 1:s1)
cost_from_stations = sum(t_vals[k,j] * d_station_landfill[k,j] * 0.5 for k in 1:s1, j in 1:m1)

total_transportation_cost = cost_direct + cost_to_stations + cost_from_stations

println("Transportation cost directly from centers to landfills: \$", cost_direct)
println("Transportation cost from centers to transfer stations: \$", cost_to_stations)
println("Transportation cost from transfer stations to landfills: \$", cost_from_stations)
println("Total transportation cost: \$", total_transportation_cost)

Transportation cost directly from centers to landfills: $404537.24764089426
Transportation cost from centers to transfer stations: $156251.50414975767
Transportation cost from transfer stations to landfills: $157854.00240069348
Total transportation cost: $718642.7541913454


In [12]:
# Extract binary variable values
z_vals = value.(z)

# Calculate operating costs of transfer stations
operating_cost = sum(10000 * z_vals[k] for k in 1:s1)

# Total daily cost
total_daily_cost = total_transportation_cost + operating_cost

println("Operating cost of transfer stations: \$", operating_cost)
println("Total daily cost: \$", total_daily_cost)

Operating cost of transfer stations: $90000.0
Total daily cost: $808642.7541913454


## Part C. A second region
### Part e)

In [13]:
# Distance matrices for Region 2
d_center_landfill = [distance(centers2[i, :], landfills2[j, :]) for i in 1:n2, j in 1:m2]
d_center_station = [distance(centers2[i, :], stations2[k, :]) for i in 1:n2, k in 1:s2]
d_station_landfill = [distance(stations2[k, :], landfills2[j, :]) for k in 1:s2, j in 1:m2]

# Total waste generated in Region 2
total_waste = sum(q2[i] for i in 1:n2)

# Create the optimization model for Region 2
model2 = Model(Gurobi.Optimizer)

# Decision variables
@variable(model2, x[j=1:m2], Bin)                # 1 if landfill j is built
@variable(model2, z[k=1:s2], Bin)                # 1 if transfer station k is built
@variable(model2, y[i=1:n2, j=1:m2] >= 0)        # Waste from center i to landfill j
@variable(model2, w[i=1:n2, k=1:s2] >= 0)        # Waste from center i to transfer station k
@variable(model2, t[k=1:s2, j=1:m2] >= 0)        # Waste from station k to landfill j

# Objective: Minimize total cost
@objective(model2, Min,
    # Direct transport from centers to landfills
    sum(y[i,j] * d_center_landfill[i,j] * 1.0 for i in 1:n2, j in 1:m2) +
    # Transport from centers to transfer stations
    sum(w[i,k] * d_center_station[i,k] * 1.0 for i in 1:n2, k in 1:s2) +
    # Transport from transfer stations to landfills
    sum(t[k,j] * d_station_landfill[k,j] * 0.5 for k in 1:s2, j in 1:m2) +
    # Operating cost of transfer stations
    sum(10000 * z[k] for k in 1:s2)
)

# Constraints

# All waste from each center must be collected
for i in 1:n2
    @constraint(model2, sum(y[i,j] for j in 1:m2) + sum(w[i,k] for k in 1:s2) == q2[i])
end

# Transfer station capacity constraints
for k in 1:s2
    @constraint(model2, sum(w[i,k] for i in 1:n2) <= 2000 * z[k])
end

# Waste flow conservation at transfer stations
for k in 1:s2
    @constraint(model2, sum(t[k,j] for j in 1:m2) == sum(w[i,k] for i in 1:n2))
end

# Landfills can only receive waste if they are built
for j in 1:m2
    @constraint(model2, sum(y[i,j] for i in 1:n2) + sum(t[k,j] for k in 1:s2) <= total_waste * x[j])
end

# Exactly 5 landfills must be built
@constraint(model2, sum(x[j] for j in 1:m2) == 5)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-19


x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] = 5

In [14]:
optimize!(model2)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 156 rows, 3415 columns and 8780 nonzeros
Model fingerprint: 0x0fe4e4ca
Variable types: 3350 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+04]
  Objective range  [1e-01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 3e+03]
Found heuristic solution: objective 5663013.4966
Presolve time: 0.01s
Presolved: 156 rows, 3415 columns, 8780 nonzeros
Variable types: 3350 continuous, 65 integer (65 binary)

Root relaxation: objective 7.972558e+05, 88 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 797255.813    0   11 5663013.50 797255.813  85.9%     -    0s
H    0     0      

In [15]:
# Extract variable values
x_vals = value.(x)
z_vals = value.(z)

# Landfills that are built
built_landfills = [j for j in 1:m2 if x_vals[j] > 0.5]
println("Landfills built at indices: ", built_landfills)

# Transfer stations that are built
built_stations = [k for k in 1:s2 if z_vals[k] > 0.5]
println("Transfer stations built at indices: ", built_stations)

Landfills built at indices: [6, 8, 9, 13, 15]
Transfer stations built at indices: [6, 7, 24, 31, 32, 36]


In [16]:

# Extract flow variables
y_vals = value.(y)
w_vals = value.(w)
t_vals = value.(t)

# Calculate distances
distance_direct = sum(y_vals[i,j] * d_center_landfill[i,j] for i in 1:n2, j in built_landfills)
distance_to_stations = sum(w_vals[i,k] * d_center_station[i,k] for i in 1:n2, k in built_stations)
distance_from_stations = sum(t_vals[k,j] * d_station_landfill[k,j] for k in built_stations, j in built_landfills)

total_distance = distance_direct + distance_to_stations + distance_from_stations

println("Distance traveled directly from centers to landfills (miles-tons): ", distance_direct)
println("Distance traveled from centers to transfer stations (miles-tons): ", distance_to_stations)
println("Distance traveled from transfer stations to landfills (miles-tons): ", distance_from_stations)
println("Total distance traveled by waste (miles-tons): ", total_distance)

Distance traveled directly from centers to landfills (miles-tons): 832002.3039437792
Distance traveled from centers to transfer stations (miles-tons): 104881.74528377625
Distance traveled from transfer stations to landfills (miles-tons): 309754.36246604245
Total distance traveled by waste (miles-tons): 1.246638411693598e6


In [17]:
# Calculate transportation costs
cost_direct = distance_direct * 1.0        # $1 per mile-ton
cost_to_stations = distance_to_stations * 1.0  # $1 per mile-ton
cost_from_stations = distance_from_stations * 0.5  # $0.5 per mile-ton

total_transportation_cost = cost_direct + cost_to_stations + cost_from_stations

println("Transportation cost directly from centers to landfills: \$", cost_direct)
println("Transportation cost from centers to transfer stations: \$", cost_to_stations)
println("Transportation cost from transfer stations to landfills: \$", cost_from_stations)
println("Total transportation cost: \$", total_transportation_cost)


# Calculate operating costs of transfer stations
operating_cost = sum(10000 * z_vals[k] for k in built_stations)

# Total daily cost
total_daily_cost = total_transportation_cost + operating_cost

println("Operating cost of transfer stations: \$", operating_cost)
println("Total daily cost: \$", total_daily_cost)

Transportation cost directly from centers to landfills: $832002.3039437792
Transportation cost from centers to transfer stations: $104881.74528377625
Transportation cost from transfer stations to landfills: $154877.18123302123
Total transportation cost: $1.0917612304605767e6
Operating cost of transfer stations: $60000.0
Total daily cost: $1.1517612304605767e6


In [18]:
# Calculate operating costs of transfer stations
operating_cost = sum(10000 * z_vals[k] for k in built_stations)

# Total daily cost
total_daily_cost = total_transportation_cost + operating_cost

println("Operating cost of transfer stations: \$", operating_cost)
println("Total daily cost: \$", total_daily_cost)

Operating cost of transfer stations: $60000.0
Total daily cost: $1.1517612304605767e6


### Part f)

In [19]:
# Distance matrices
d_center_landfill = [distance(centers_all[i, :], landfills_all[j, :]) for i in 1:n_all, j in 1:m_all]
d_center_station = [distance(centers_all[i, :], stations_all[k, :]) for i in 1:n_all, k in 1:s_all]
d_station_landfill = [distance(stations_all[k, :], landfills_all[j, :]) for k in 1:s_all, j in 1:m_all]

model_combined = Model(Gurobi.Optimizer)
# Decision variables
@variable(model_combined, x[j=1:m_all], Bin)             # 1 if landfill j is built
@variable(model_combined, z[k=1:s_all], Bin)             # 1 if transfer station k is built
@variable(model_combined, y[i=1:n_all, j=1:m_all] >= 0)  # Waste from center i to landfill j
@variable(model_combined, w[i=1:n_all, k=1:s_all] >= 0)  # Waste from center i to transfer station k
@variable(model_combined, t[k=1:s_all, j=1:m_all] >= 0)  # Waste from station k to landfill j
# Objective: Minimize total cost
@objective(model_combined, Min,
    # Direct transport from centers to landfills
    sum(y[i,j] * d_center_landfill[i,j] * 1.0 for i in 1:n_all, j in 1:m_all) +
    # Transport from centers to transfer stations
    sum(w[i,k] * d_center_station[i,k] * 1.0 for i in 1:n_all, k in 1:s_all) +
    # Transport from transfer stations to landfills
    sum(t[k,j] * d_station_landfill[k,j] * 0.5 for k in 1:s_all, j in 1:m_all) +
    # Operating cost of transfer stations
    sum(10000 * z[k] for k in 1:s_all)
)

for i in 1:n_all
    @constraint(model_combined, sum(y[i,j] for j in 1:m_all) + sum(w[i,k] for k in 1:s_all) == q_all[i])
end

for k in 1:s_all
    @constraint(model_combined, sum(w[i,k] for i in 1:n_all) <= 2000 * z[k])
end

for k in 1:s_all
    @constraint(model_combined, sum(t[k,j] for j in 1:m_all) == sum(w[i,k] for i in 1:n_all))
end

total_waste = sum(q_all)  # Total waste generated in the combined region

for j in 1:m_all
    @constraint(model_combined, sum(y[i,j] for i in 1:n_all) + sum(t[k,j] for k in 1:s_all) <= total_waste * x[j])
end
@constraint(model_combined, sum(x[j] for j in 1:m_all) <= 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-19


x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] + x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] ≤ 10

In [20]:
optimize!(model_combined)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 321 rows, 14830 columns and 38560 nonzeros
Model fingerprint: 0x9d7941d5
Variable types: 14700 continuous, 130 integer (130 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+03]
Found heuristic solution: objective 1.416122e+07
Presolve time: 0.02s
Presolved: 321 rows, 14830 columns, 38560 nonzeros
Variable types: 14700 continuous, 130 integer (130 binary)

Root relaxation: objective 1.443415e+06, 211 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1443415.39    0   24 1.4161e+07 1443415.39  89.8%     -    0s
H    0 

In [21]:
# Extract variable values
x_vals = value.(x)
z_vals = value.(z)

# Landfills that are built
built_landfills = [j for j in 1:m_all if x_vals[j] > 0.5]
println("Landfills built at indices: ", built_landfills)

# Transfer stations that are built
built_stations = [k for k in 1:s_all if z_vals[k] > 0.5]
println("Transfer stations built at indices: ", built_stations)

Landfills built at indices: [2, 3, 14, 16, 17, 21, 23, 24, 28, 30]
Transfer stations built at indices: [1, 4, 6, 11, 12, 30, 32, 39, 45, 46, 48, 57, 74, 77, 82]


In [22]:
# Extract flow variables
y_vals = value.(y)
w_vals = value.(w)
t_vals = value.(t)

# Calculate distances
distance_direct = sum(y_vals[i,j] * d_center_landfill[i,j] for i in 1:n_all, j in built_landfills)
distance_to_stations = sum(w_vals[i,k] * d_center_station[i,k] for i in 1:n_all, k in built_stations)
distance_from_stations = sum(t_vals[k,j] * d_station_landfill[k,j] for k in built_stations, j in built_landfills)

total_distance = distance_direct + distance_to_stations + distance_from_stations

println("Distance traveled directly from centers to landfills (miles-tons): ", distance_direct)
println("Distance traveled from centers to transfer stations (miles-tons): ", distance_to_stations)
println("Distance traveled from transfer stations to landfills (miles-tons): ", distance_from_stations)
println("Total distance traveled by waste (miles-tons): ", total_distance)

Distance traveled directly from centers to landfills (miles-tons): 1.1896914177799309e6
Distance traveled from centers to transfer stations (miles-tons): 232427.85365812026
Distance traveled from transfer stations to landfills (miles-tons): 586530.464006059
Total distance traveled by waste (miles-tons): 2.00864973544411e6


In [23]:
# Calculate transportation costs
cost_direct = distance_direct * 1.0        # $1 per mile-ton
cost_to_stations = distance_to_stations * 1.0  # $1 per mile-ton
cost_from_stations = distance_from_stations * 0.5  # $0.5 per mile-ton

total_transportation_cost = cost_direct + cost_to_stations + cost_from_stations

println("Transportation cost directly from centers to landfills:\$", cost_direct)
println("Transportation cost from centers to transfer stations: \$", cost_to_stations)
println("Transportation cost from transfer stations to landfills: \$", cost_from_stations)
println("Total transportation cost: \$", total_transportation_cost)

Transportation cost directly from centers to landfills:$1.1896914177799309e6
Transportation cost from centers to transfer stations: $232427.85365812026
Transportation cost from transfer stations to landfills: $293265.2320030295
Total transportation cost: $1.7153845034410805e6


In [24]:
# Calculate operating costs of transfer stations
operating_cost = sum(10000 * z_vals[k] for k in built_stations)

# Total daily cost
total_daily_cost = total_transportation_cost + operating_cost

println("Operating cost of transfer stations: \$", operating_cost)
println("Total daily cost: \$", total_daily_cost)

Operating cost of transfer stations: $150000.0
Total daily cost: $1.8653845034410805e6


### Recommendation:

Based on the analysis, I recommend using the **combined model** for managing waste in both Region 1 and Region 2. The combined strategy results in a **lower total transportation cost** (\$1,715,384) and **smaller total distance traveled** (2,008,649 miles-tons) compared to handling the regions separately, where the total transportation cost is \$1,810,403 and the distance traveled is 2,123,134 miles-tons. Additionally, the **total daily cost** for the combined model is **\$1,865,384**, which is lower than the total cost of managing the regions separately (\$1,960,403). This ensures more efficient use of resources, reduced logistical complexity, and improved overall operational efficiency.